## Try requests and bs4

In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
app_url = 'https://finance.yahoo.com/quote/APP/holders/'

In [2]:
agent = 'Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

In [4]:
response = requests.get(app_url, headers={'User-Agent': agent})
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [14]:
# Locate the section containing the "Major Holders" table by the section's `data-testid` attribute
section = soup.find('section', {'data-testid': 'holders-major-holders-table'})

In [11]:
# Find all 'td' elements within the section to extract the data
td_elements = section.find_all('td', class_='majorHolders')
td_elements

[<td class="majorHolders yf-1toamfi">27.69% </td>,
 <td class="majorHolders yf-1toamfi">% of Shares Held by All Insider </td>,
 <td class="majorHolders yf-1toamfi">60.85% </td>,
 <td class="majorHolders yf-1toamfi">% of Shares Held by Institutions </td>,
 <td class="majorHolders yf-1toamfi">84.15% </td>,
 <td class="majorHolders yf-1toamfi">% of Float Held by Institutions </td>,
 <td class="majorHolders yf-1toamfi">861 </td>,
 <td class="majorHolders yf-1toamfi">Number of Institutions Holding Shares </td>]

In [15]:
# Initialize an empty dictionary
holders_dict = {}

# Iterate through the list in pairs (numeric value and its corresponding label)
for i in range(0, len(td_elements), 2):
    # Strip any extra characters and clean up the value
    value = td_elements[i].text.strip().replace('%', '')
    
    # Convert the value to float or integer as appropriate
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    
    # Get the corresponding key (label)
    label = td_elements[i+1].text.strip()
    
    # Update the dictionary
    holders_dict[label] = value

In [17]:
holders_dict

{'% of Shares Held by All Insider': 27.69,
 '% of Shares Held by Institutions': 60.85,
 '% of Float Held by Institutions': 84.15,
 'Number of Institutions Holding Shares': 861}